In [ ]:
from lbhb.psychometric import CachedStanModel
import pandas as pd
import numpy as np
import arviz as az

In [ ]:
sig = pd.read_csv('psth_sig_cellids.csv')['cellid'].unique()
df = pd.read_csv('frequency_tuning_curves_for_bburan.csv')
df.columns = [s.replace(' ', '') for s in df.columns]
cols = ['pupil', 'frequency', 'ftc_count', 'ftc_time', 'spont_count', 'spont_time']
df = pd.wide_to_long(df, cols, 'cellid', 'idx', sep='_').dropna()
df['frequency'] = np.log(df['frequency'])

mask = df.apply(lambda x: x.name[0] in sig, axis=1)
df = df.loc[mask]

sr = df.groupby(['cellid', 'pupil'])[['spont_count', 'spont_time']].first().sort_index()
ftc = df.reset_index().set_index(['cellid', 'pupil', 'frequency'])[['ftc_count', 'ftc_time']].sort_index()
m = ftc['ftc_time'] > 0
ftc = ftc.loc[m]

In [ ]:
model = CachedStanModel('gaussian_FTC_simple.stan')
cells = ftc.index.get_level_values('cellid').unique()
pupil = 1

e = ftc.xs(pupil, level='pupil').reset_index()
s = sr.xs(pupil, level='pupil').reset_index()
#start_cell, end_cell = cells[0], cells[10]
#e = ftc.loc[start_cell:end_cell].xs(pupil, level='pupil').reset_index()
#s = sr.loc[start_cell:end_cell].xs(pupil, level='pupil').reset_index()

cells = e['cellid'].unique()
cell_map = {c: i+1 for i, c in enumerate(cells)}
e['cell_index'] = e['cellid'].apply(cell_map.get).values
s['cell_index'] = s['cellid'].map(cell_map.get)
s.sort_values('cell_index', inplace=True)

n = len(e)
frequency = e['frequency'].values
spike_count = e['ftc_count'].values.astype('i')
cell = e['cell_index'].values.astype('i')
n_cells = len(cells)
sample_time = e['ftc_time'].values
spont_count = s['spont_count'].values.astype('i')
spont_time = s['spont_time'].values

#import functools
#initer = functools.partial(init_values, e, s, n_cells)
#
#def init_values(e, s, n_cells, chain_id):
#    print(f'INITIALIZING VALUES FOR CHAIN {chain_id}')
#    #bf_start = e.groupby('cellid')['frequency'].mean().values 
#    #bw_start = e.groupby('cellid')['frequency'].agg(['min', 'max']) \
#    #    .rename(columns={'min': 'lb', 'max': 'ub'}) \
#    #    .eval('(ub-lb)/4').values
#    #y = s.set_index('cellid').eval('spont_count/spont_time')
#    #x = e.set_index('cellid').eval('ftc_count/ftc_time')
#    #sr_start = y.values
#    #gain_start = (x-y).groupby('cellid').apply(lambda x: np.abs(x).min()).values
#    
#    return {
#        'bf': bf_start,
#        'bandwidth': bw_start,
#        'offset': sr_start,
#        'gain': gain_start,
#    }

data = {
    'n': n,
    'n_cells': n_cells,
    'cell': cell,
    'freq': frequency,
    'spike_count': spike_count,
    'sample_time': sample_time,
    'spont_count': spont_count,
    'spont_time': spont_time,
}

fit = model.sampling(data, control={'adapt_delta': 0.99, 'max_treedepth': 20})

In [ ]:
import numpy as np
import pylab as pl

def atleast_2d(x):
    if x.ndim == 1:
        return x[..., np.newaxis]
    else:
        return x

f, axes = pl.subplots(5, 5, figsize=(10, 10), squeeze=False)

for cell, ax in zip(cells, axes.ravel()):
    i = cell_map[cell] - 1

    bf = atleast_2d(fit['bf']).mean(axis=0)[i]
    gain = atleast_2d(fit['gain']).mean(axis=0)[i]
    bw = atleast_2d(fit['bandwidth']).mean(axis=0)[i]
    offset = atleast_2d(fit['offset']).mean(axis=0)[i]
    
    print(f'BF {bf:.2f}, gain {gain:.2f}, bw {bw:.2f}, offset {offset:.2f}')

    frequency = np.arange(3, 11, 0.1)
    d = ftc.loc[cell, 1].reset_index()
    s = sr.loc[cell, 1]
    l = np.exp(-0.5*np.square((frequency-bf)/bw))
    l = offset + gain * l

    ax.plot(frequency, l, 'k-')
    ax.plot(d['frequency'], d.eval('ftc_count/ftc_time'), 'go-')
    ax.axhline(s['spont_count']/s['spont_time'])
    
az.plot_trace(fit, ['offset'])

In [ ]:
ftc.reset_index()['frequency'].describe()

In [ ]:
fit['bandwidth'].mean(axis=0).mean()
fit['bandwidth_mean'].mean(), fit['bandwidth_sd'].mean()

In [ ]:
av.plot_trace(fit, ['bandwidth_mean', 'bandwidth_sd', 'offset_alpha', 'offset_beta'])